In [1]:
import io
import re
from collections import Counter

import mysql.connector
import pdfplumber
import requests
from bs4 import BeautifulSoup
from spacy import load

In [62]:
# Print iterations progress
# source: https://stackoverflow.com/questions/3173320/text-progress-bar-in-terminal-with-block-characters?noredirect=1&lq=1
def print_progress_bar(iteration, total, prefix='', suffix='Complete', decimals=1, length=100, fill='█', printEnd="\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end=printEnd)
    # Print New Line on Complete
    if iteration == total:
        print()


def get_protocol_link_from_website(url: str):
    # https://www.parlament.gv.at/PAKT/VHG/XXVII/NRSITZ/NRSITZ_00113/#tab-Sten.Protokoll
    content = requests.get(url).content.decode('utf-8')
    soup = BeautifulSoup(content, 'html.parser')
    anchors = soup.find_all('a', href=True)
    for anchor in anchors:
        if len(anchor.contents) >= 2:
            if 'Stenographisches Protokoll' in anchor.contents[1]:
                return 'https://www.parlament.gv.at' + anchor['href']


def get_text_from_online_pdf(url: str):
    """Extract text from a pdf from a given url. Returns a string"""

    response = io.BytesIO(requests.get(url).content)
    return get_text_from_bytes(response)


def get_text_from_bytes(bytes: io.BytesIO):
    """Extract text from a pdf which is in a BytesIO format. Returns a string"""

    text = ''

    with pdfplumber.open(bytes) as pdf:
        for page in pdf.pages:
            #print(page.page_number)
            print_progress_bar(page.page_number, len(pdf.pages), 'Extracting text:')
            text += page.extract_text()
    return text


def get_speech_positions(text: str):
    """Gets all positions of speeches in the text. A speech is defined as the text between two time specifications."""
    matches = re.finditer('[0-2]?[0-9]\.[0-5][0-9] \n', text)
    matches_list = [item for item in matches]
    # TODO: muss da -1?
    positions = [(matches_list[idx].end(), matches_list[idx + 1].start()) for idx in range(0, len(matches_list) - 1, 2)]
    return positions


def prepare_text(text: str, positions: list((int, int))):
    all_text = ''
    # Load spacy model for stopwords
    nlp = load('de_core_news_sm')
    for pos in positions:
        # Get speech between the specified time
        speech = text[pos[0]:pos[1]]

        # Remove headers
        speech = re.sub(' \d*? .* Nationalrat, ?.* ?\n?.*', '', speech)
        speech = re.sub('Nationalrat, ?.* ?\n?.*', '', speech)

        # Merge words separated by a line-break
        speech = re.sub('- ?\n', '', speech)
        speech = re.sub('\n', '', speech)

        # Remove the beginnging of the string e.g. "Abgeordneter Jon Doe (Blue party):"
        speech = re.sub('Abgeordneter \D*: ', '', speech)
        speech = re.sub('Abgeordnete \D*: ', '', speech)

        # Remove stop words
        result = [word for word in speech.split() if word not in nlp.Defaults.stop_words]
        speech = ' '.join(result)

        # Remove punctuation
        speech = re.sub(r'[^\w\s]+', '', speech)

        all_text += speech

    return all_text


def save_to_database(word_counts: dict, db_connection: mysql.connector):
    if len(list(word_counts.items())) <= 0:
        return

    cursor = db_connection.cursor()

    cursor.execute('SELECT * FROM WORD_COUNTS')
    result = dict(cursor.fetchall())

    for word in word_counts:
        if word in list(result.keys()):
            result[word] += word_counts[word]
        else:
            result[word] = word_counts[word]

    cursor.execute('TRUNCATE TABLE WORD_COUNTS')

    sql = 'INSERT INTO WORD_COUNTS (WORD, COUNT) VALUES (%s, %s)'
    values = list(word_counts.items())
    #print(values)
    cursor.executemany(sql, values)

    db.commit()

    print(cursor.rowcount, "lines were inserted")

In [ ]:
db = mysql.connector.connect(
    host="localhost",
    user="htl",
    password="insy",
    database='politics'
)
cursor = db.cursor()
#cursor.execute('TRUNCATE TABLE WORD_COUNTS')
#url = 'https://www.parlament.gv.at/PAKT/VHG/XXVII/NRSITZ/NRSITZ_00113/fname_1009194.pdf'

# inkl. 0; exkl. 107
for idx in range(1, 107):
    # fill with zeros
    nr = str(idx).zfill(5)
    url = f'https://www.parlament.gv.at/PAKT/VHG/XXVII/NRSITZ/NRSITZ_{nr}/#tab-Sten.Protokoll'

    link = get_protocol_link_from_website(url)
    text = get_text_from_online_pdf(link)
    positions = get_speech_positions(text)
    all_text = prepare_text(text, positions)
    c = Counter([word.strip() for word in all_text.split()])
    #print(dict(c))
    save_to_database(dict(c), db)
    break

# ich 76

Extracting text: |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete


db = mysql.connector.connect(
    host="localhost",
    user="htl",
    password="insy",
    database='politics'
)

cursor.execute('SELECT * FROM WORD_COUNTS')
result = dict(cursor.fetchall())
print(result)
add = {'Ich': 4, 'du': 1}
for item in add:
    if item in list(result.keys()):
        result[item] += add[item]
    else:
        result[item] = add[item]
#print(list(result.keys()))
#add['Ich'] = add['Ich'] + 4

print(result)
#print(result)

db = mysql.connector.connect(
    host="localhost",
    user="htl",
    password="insy",
    database='politics'
)
cursor = db.cursor()
cursor.execute('TRUNCATE TABLE WORD_COUNTS')
cursor.execute('INSERT INTO WORD_COUNTS (WORD, COUNT) VALUES ("ich", 3)')
cursor.execute('INSERT INTO WORD_COUNTS (WORD, COUNT) VALUES ("Ich", 5)')
db.commit()

cursor = db.cursor()

cursor.execute('DROP TABLE WORD_COUNTS;')
cursor.execute('CREATE TABLE WORD_COUNTS (WORD VARCHAR(255) CHARACTER SET utf8 COLLATE utf8_bin NOT NULL UNIQUE, COUNT INTEGER);')
#cursor.execute('TRUNCATE TABLE WORD_COUNTS')
db.commit()